In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install dabl

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dabl
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

# EDA

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.drop(['PassengerId', 'Name'], axis=1, inplace=True)

In [ ]:
train_df.describe().T

In [ ]:
sns.countplot(x=train_df['Survived'], palette='pastel')

In [ ]:
def make_countplot(train_df, col):
    sns.countplot(x=train_df['Survived'], palette='pastel', hue=train_df[col])
    plt.tight_layout(rect=(0, 0, 1.2, 1.2))
    plt.show()
#     plt.tight_layout(rect=(0, 0, 1.2, 1.2))
    return None

In [ ]:
df = train_df.drop(['Age', 'Fare', 'Ticket', 'Cabin'], axis=1)
for col in df.columns:
    make_countplot(df, col)


In [ ]:
df = train_df

In [ ]:
plt.pie(df['Sex'].value_counts(), labels=['Male', 'Female'], autopct='%.2f')
plt.axis('equal')
plt.show()

In [ ]:
sns.violinplot(x=train_df['Fare'])
sns.boxplot(x=train_df['Fare'], color='orange')
plt.show()
sns.histplot(x=train_df['Fare'], bins=20, kde=True, color='purple')

In [ ]:
pp = sns.pairplot(df, size=1.8, aspect=1.8, diag_kind='kde', diag_kws=dict(shade=True))
fig = pp.fig
fig.subplots_adjust(top=0.83, wspace=0.33)

In [ ]:
sns.jointplot(x=df['Fare'], y=df['Age'])

In [ ]:
df[df['Fare'] > 200]

In [ ]:
idx = df[df['Fare'] > 500].index 
df.drop(idx, inplace=True)

In [ ]:
idx = df[df['Age'] < 1].index
df.drop(idx, inplace=True)

In [ ]:
dabl.plot(df.drop('Survived', axis=1), df['Survived'])

### 0 -> did not survive
### 1 -> survived

# Imputation

In [ ]:
df.isna().sum()[df.isna().sum() != 0]

In [ ]:
df['Cabin'].value_counts().sum()

there are only 202 filled values in Cabin column, 686 values are NaN values

In [ ]:
df.drop('Cabin', axis=1, inplace=True)

In [ ]:
imputer = KNNImputer()
x = imputer.fit_transform(df.select_dtypes(exclude='object'))
df_filled = pd.DataFrame(data=x, columns=df.select_dtypes(exclude='object').columns)

In [ ]:
df_filled.head()

In [ ]:
df = df.reset_index()

In [ ]:
df = pd.concat([df_filled, df.select_dtypes(include='object')], axis=1)

In [ ]:
df

In [ ]:
df['Embarked'].ffill(inplace=True)

In [ ]:
df.isna().sum()

# Encoding categorical columns

In [ ]:
df.select_dtypes(include='object').columns

In [ ]:
print('Column "Sex" has ' + str(df['Sex'].nunique()) + ' unique values')
print('Column "Ticker" has ' + str(df['Ticket'].nunique()) + ' unique values')
print('Column "Embarked" has ' + str(df['Embarked'].nunique()) + ' unique values')

In [ ]:
df.drop('Ticket', axis=1, inplace=True)

In [ ]:
df = pd.concat([pd.get_dummies(df['Sex'], drop_first=True) , df.drop('Sex', axis=1)], axis=1)
df = pd.concat([pd.get_dummies(df['Embarked'], drop_first=True) , df.drop('Embarked', axis=1)], axis=1)

In [ ]:
df.head()

# Train-Test split of dataset

In [ ]:
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [ ]:
X.head()

# Preparing Test data for submission

In [ ]:
test_df

In [ ]:
test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_df.head()

In [ ]:
test_df = pd.concat([pd.get_dummies(test_df['Sex'], drop_first=True) , test_df.drop('Sex', axis=1)], axis=1)
test_df = pd.concat([pd.get_dummies(test_df['Embarked'], drop_first=True) , test_df.drop('Embarked', axis=1)], axis=1)

In [ ]:
test_df.head()

# Create and Train ML model

In [ ]:
xgb = XGBClassifier()
rf = RandomForestClassifier(n_estimators=30, max_depth=30, oob_score=True)
knn = KNeighborsClassifier(n_neighbors=10)

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"The model's score is: {score*100:.2f}%")
    print(classification_report(y_test, model.predict(X_test)))
    print(confusion_matrix(y_test, model.predict(X_test)))

In [ ]:
train_model(xgb, X_train, y_train, X_test, y_test)

In [ ]:
train_model(rf, X_train, y_train, X_test, y_test)

In [ ]:
train_model(knn, X_train, y_train, X_test, y_test)

In [ ]:
print(f"Column names: {X.columns}")
xgb.feature_importances_

In [ ]:
sns.violinplot(x=df['Survived'], y=df['male'])

# Clearly, Random Forest Classifier performed the best. So we will use this for our predictions

## DL model

In [ ]:
X_train.shape

In [ ]:
model = Sequential()

model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
early_stop = EarlyStopping(patience=5, mode='min', monitor='val_loss')

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stop], validation_data=(X_test, y_test))

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
model.predict(X_test)

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
test_df.head()

In [ ]:
predictions = xgb.predict(test_df)

In [ ]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")